In [7]:
import pandas as pd
import re
issue_db = pd.read_csv('issueDB.csv')

In [11]:
# get the max number of letters in issue title
print(issue_db['Title'].str.len().max())
print("A"*10000)

587.0
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [4]:
def clean_text(issue_content):
    if pd.isna(issue_content): return ' '
    issue_content = re.sub(r'\n', ' ', issue_content)
    # Remove code block ``` xxx ```
    issue_content = re.sub(r'```.*?```', ' ', issue_content, flags=re.DOTALL)
    # Remove hyperlink [xxx](xxx)
    issue_content = re.sub(r'\[.*?\]\(.*?\)', ' ', issue_content)
    # Remove hyperlink ![xxx](xxx)
    issue_content = re.sub(r'\!\[.*?\]\(.*?\)', ' ', issue_content)
    # Remove URL http://xxx
    issue_content = re.sub(r'http\S+', ' ', issue_content)
    # Remove XML tag <xxx>
    issue_content = re.sub(r'<.*?>', ' ', issue_content)
    return issue_content

In [ ]:
up = 20000 # Contain
limit = 20000 # Not contain
issue_title = issue_db['Title'][up:limit].copy()
issue_body = issue_db['Body'][up:limit].copy()
assert len(issue_title) == len(issue_body)
for i in range(up, up + len(issue_title)):
    issue_title[i] = clean_text(issue_title[i])
    issue_body[i] = clean_text(issue_body[i])
print(len(issue_title))
print(len(issue_db))
print(issue_db.loc[[up]])
issue_title.to_csv(f'/Users/home/Documents/PyCharm/Senti4SD/ClassificationTask/title{up}-.csv', header=False,index=False)
issue_body.to_csv(f'/Users/home/Documents/PyCharm/Senti4SD/ClassificationTask/body{up}-.csv', header=False,index=False)

In [ ]:
prediction = pd.read_csv('/Users/home/Desktop/sentiment/title/predictions20000-.csv')
print(len(prediction))
#delete the 1st letter of the 'Row' column and convert it to int
prediction['Row'] = prediction['Row'].str[1:].astype(int)
#sort the dataframe by the 'Row' column
prediction = prediction.sort_values(by=['Row'])
print(prediction)
#output the dataframe to a csv file
prediction.to_csv('/Users/home/Desktop/sentiment/title/5.csv', index=False, header=False)

In [ ]:
# Concatinate all the csv files 1 2 3 4 5
pd.concat([pd.read_csv('/Users/home/Desktop/sentiment/title/1.csv',header=None),
           pd.read_csv('/Users/home/Desktop/sentiment/title/2.csv',header=None),
           pd.read_csv('/Users/home/Desktop/sentiment/title/3.csv',header=None),
           pd.read_csv('/Users/home/Desktop/sentiment/title/4.csv',header=None),
           pd.read_csv('/Users/home/Desktop/sentiment/title/5.csv',header=None)]).to_csv('/Users/home/Desktop/sentiment/title/predictions-title.csv', index=False)

In [ ]:
all_prediction = pd.read_csv('/Users/home/Desktop/sentiment/title/predictions-title.csv')
print(len(all_prediction))
print(len(issue_db))
assert len(all_prediction) == len(issue_db)
# delete the 1st column
all_prediction = all_prediction.drop(columns=['0'])
print(all_prediction)
# append it to the issue_db
issue_db['Sentiment_Title'] = all_prediction
print(issue_db.tail())

In [ ]:
# Output the issue_db to a csv file
issue_db = pd.read_csv('issueDB.csv')
issue_db['Sentiment_Title'] = all_prediction
#issue_db.to_csv('issueDB.csv', index=False)